In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Modifying Solver Options

In OpenMDAO, implicit outputs can have bounds and scaling attached to them. These are defined when `add_output` is called on an implicit component.

In complex models, this means that a user may need to tweak bounds and scaling in multiple files. The bounds and scaling of a variable may be dependent upon the application of the model. These bounds may need to be adjusted in a file that is generally outside of the expertise of the user.

OpenMDAO provides a method `System.set_output_solver_options` that allows users to set bounds throughout their model from a single script or notebook.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.set_output_solver_options
        :noindex:
```

## Example of Setting Solver Output Options

Here is an example of how to make use of the `System.set_output_solver_options` method.

In [ ]:
# Implicit component in which bounds and scaling are set on outputs
import openmdao.api as om

class ScalingExample3(om.ImplicitComponent):

    def setup(self):
        self.add_input('x1', val=100.0)
        self.add_input('x2', val=5000.0)
        self.add_output('y1', val=200., ref=1e2, res_ref=1e5, ref0=1.0, lower=2., upper=3)
        self.add_output('y2', val=6000., ref=1e3, res_ref=1e-5)

    def apply_nonlinear(self, inputs, outputs, residuals):
        x1 = inputs['x1']
        x2 = inputs['x2']
        y1 = outputs['y1']
        y2 = outputs['y2']

        residuals['y1'] = 1e5 * (x1 - y1) / y1
        residuals['y2'] = 1e-5 * (x2 - y2) / y2

In [ ]:
# Script that makes use of this implicit component
prob = om.Problem()

# ScalingExample3 sets values for ref, res_ref, ref0, lower, and upper
comp = prob.model.add_subsystem('comp', ScalingExample3())

prob.setup() 
prob.run_model() # Run model with the bounds and scaling set in the component

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
with prob.model._scaled_context_all():
    assert_near_equal(prob.get_val('comp.y1'), 2.0, 1e-2)

In [ ]:
# Now override a value from this cell (could also be from a separate script) and re-run the model
prob.model.set_output_solver_options(name='comp.y1',ref=1e3)
prob.setup() 
prob.run_model() 

In [ ]:
with prob.model._scaled_context_all():
    assert_near_equal(prob.get_val('comp.y1'), 0.2, 1e-2)